In [0]:
# encoding=utf-8
# https://github.com/zhanlaoban/EDA_NLP_for_Chinese
import jieba
import synonyms
import random
from random import shuffle

In [0]:
import pandas as pd
from tqdm import tqdm

In [0]:
import os
path = "/content/drive/My Drive/NLP/sentiment_compete"
os.chdir(path)

In [7]:
!pip install -U synonyms

Requirement already up-to-date: synonyms in /usr/local/lib/python3.6/dist-packages (3.10.2)


In [0]:
#停用词列表，默认使用哈工大停用词表
f = open('stop_words/hit_stopwords.txt')
stop_words = list()
for stop_word in f.readlines():
    stop_words.append(stop_word[:-1])

In [0]:
def synonym_replacement(words, replace_num):
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words])) # 非停用词
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms_words = synonyms.nearby(random_word)[0]
        if len(synonyms_words) >= 1:
            synonyms_word = random.choice(synonyms_words[1: min(5, len(synonyms_words))])
            new_words = [synonyms_word if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= replace_num:
            break

    new_sentence = ''.join(new_words)
    return new_sentence

In [0]:
def sentence_augument(sentence, num_aug = 2, alpha_sy = 0.1):
    seg_list = jieba.cut(sentence)
    seg_list = " ".join(seg_list)
    words = list(seg_list.split())
    num_words = len(words)
    replace_num = max(1, int(num_words *  alpha_sy))
    augmented_sentences = []
    for _ in range(num_aug):
        augmented_sentences.append(synonym_replacement(words, replace_num))
    return augmented_sentences

In [0]:
def data_augument(df, num_aug = 2):
    df['微博中文内容'] = df['微博中文内容'].astype(str)
    for index, line in tqdm(df.iterrows()):
        augument_sentences = sentence_augument(line['微博中文内容'], num_aug)
        for augment_line in augument_sentences:
            new_line = line.copy()
            new_line['微博中文内容'] = augment_line
            df = df.append(new_line, ignore_index=True)
    return df

In [0]:
text = '今天是个好日子,心想的事儿都能成,马上冲进top10'

In [13]:
data_augument(text)

TypeError: ignored

In [0]:
train_data = pd.read_csv('data/train_weibo.csv')

In [15]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81098 entries, 0 to 81097
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   微博id    81098 non-null  int64 
 1   微博发布时间  81098 non-null  object
 2   发布人账号   81098 non-null  object
 3   微博中文内容  80262 non-null  object
 4   微博图片    81098 non-null  object
 5   微博视频    81098 non-null  object
 6   情感倾向    81098 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 4.3+ MB


In [16]:
train_data['情感倾向'].value_counts()

 0    46163
 1    20397
-1    14538
Name: 情感倾向, dtype: int64

In [0]:
train_data_1 = train_data[train_data['情感倾向'] == 1]

In [18]:
train_data_1= data_augument(train_data_1, 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
20397it [19:01, 17.87it/s]


In [19]:
train_data_1.shape

(40794, 7)

In [20]:
train_data_neg = train_data[train_data['情感倾向'] == -1]
train_data_neg= data_augument(train_data_neg, 2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
14538it [13:45, 17.61it/s]


In [0]:
train_data_0 = train_data[train_data['情感倾向'] == 0]

In [0]:
train_aug = pd.concat([train_data_0, train_data_neg, train_data_1], ignore_index=True)

In [0]:
from sklearn.utils import shuffle
train_aug = shuffle(train_aug)

In [24]:
train_aug['情感倾向'].value_counts()

 0    46163
-1    43614
 1    40794
Name: 情感倾向, dtype: int64

In [0]:
train_aug.to_csv('data/train_weibo_aug.csv', index=False)